# Exercise set 3
Instructions: Use Google Colab to run the notebook; click the rocket-icon and choose Colab. Submit the completed notebook as instructed in the course.

# Logistic regression
We consider two-dimensional data classification onto two classes with a logistic regression model.

The logistic regression model with parameters $\theta_1$, $\theta_2$, and $T$, classifies a datapoint $(x_1, x_2)$ 
onto class 'positive' ($Y=1$) or class 'negative' ($Y=0$):
* 'positive': $p(Y=1 | x_1, x_2, \theta_1, \theta_2)\geq T$,
* 'negative': $p(Y=1 | x_1, x_2, \theta_1, \theta_2) < T$,

where $p(Y=y | x_1, x_2, \theta_1, \theta_2) = \rm{Bernoulli}(y; \text{logistic_sigmoid}\left(\theta_1 x_1+\theta_2 x_2 \right))$, 
and so $p(Y=1 | x_1, x_2, \theta_1, \theta_2)=\left(1+\exp{\left(-\theta_1 x_1-\theta_2 x_2 \right)}\right)^{-1}$. 

The decision boundary can be then mathematically expressed with the equation $\left(1+\exp{\left(-\theta_1 x_1-\theta_2 x_2 \right)}\right)^{-1}=T$, where the classification threshold $T\in (0, 1)$.

With a bit of algebra, we can obtain that $
x_2 = -\frac{\theta_1}{\theta_2}x_1-\frac{\ln{\left(\frac{1-T}{T}\right)}}{\theta_2}=-\frac{\theta_1}{\theta_2}x_1+\frac{\ln{\left(\frac{T}{1-T}\right)}}{\theta_2}$,
so we can see that the boundary is a line where $-\frac{\theta_1}{\theta_2}$ is slope and $\frac{\ln{\left(\frac{T}{1-T}\right)}}{\theta_2}$ intercept.

In the notebook below, we: 

1. create a dataset with 2D-data coming from two different classes ('positive', 'negative')
2. split the dataset onto three sets: training, validation, and test
3. optimize model parameter values using an optimization scheme
4. test (& tweak) the optimized model 

Your task is to study the procedure, the effect of the parameters that are varied using the interactive controls, and answer
the questions.

## Task 0: get and import needed helper code 

In [ ]:
!git clone https://github.com/luma-lapinamk/jyri-pso23.git

In [ ]:
import sys
sys.path.insert(0, 'jyri-pso23/code')
from helpers_set3 import *

## Task 1: Dataset creation
Executing the cell below creates an interactive plot to create a daset. The probability density of values for data coming from a class is represented with a bivariate normal (Gaussian) distribution with "spherical covariance".  

Properties that can be interactively changed:
* num_positives: Amount of examples from class 'positive' (triangles, red)
* num_negatives: Amount of examples from class 'negative' (circles, blue)
* positives_mean: Mean vector of the Gaussian for the positive class
* positives_scale: Standard deviations of the Gaussian for the positive class; same along both dimensions, no correlation 
* negatives_mean: Mean vector of the Gaussian for the negative class
* negatives_scale: Standard deviations of the Gaussian for the negative class; same along both dimensions, no correlation

Press the 'Run Interact'-button to create (sample) and plot a dataset; note that the results change for different runs with fixed interaction parameter values, a different data sampling for each run. Study the interaction parameters and when varying them, think about the case of classification that is faced, decision boundary that is to separate the classes is a line. Construct a dataset for which there are expected to be clear classification errors, but also for which the classification is expected to be able to be clearly better than the method of randomly guessing. Once you are happy with the dataset, execute also the cell below it to extract the data from the interactive plot instance.

In the 'Answer'-section, briefly describe in the empty cell i) how should the amount of examples (samples) contribute to the classification performance of new, unseen data from the sources, ii) the effect of considering mistakes under a class being worse than in the other, to the classifier properties, and iii) the parameter values that you chose.

In [ ]:
interactive_plot = setup_dataset_interactively()
interactive_plot

In [ ]:
positives, num_positives, negatives, num_negatives = interactive_plot.result # extract the data

### Answer
Input your answers to the cell below.

## Task 2: Dataset splitting for development and testing
Executing the cell below creates an interactive plot to split the daset onto training, validation, and test portions.

Properties that can be interactively changed:
* split_ratio_dev: Rate of the total amount training and validation examples to the amount of test examples (num_train+num_validation:num_test)
* split_ratio_trainval: Rate of the amount of training examples to the amount of validation examples (num_train:num_validation)

Press the 'Run Interact'-button to create the splits, and also to create a visualization; note that the results do not change for a different run with the same interaction parameter values. In the visualization one visualizes the objective function that is used in gradient-based optimization of model parameter values ($\text{parameter 1}=\theta_1$, 
$\text{parameter 2}=\theta_2$), under the training data and under the validation data; the objective function is the negative joint conditional log-likelihood of data under the model: $-\sum_{n=1}^{N}\log p(Y=y^{(n)}|x_{1}^{(n)}, x_{2}^{(n)}, \theta_1, \theta_2)$, where $(x_{1}^{(n)}, x_{2}^{(n)})$ are the coordinates of the n$^{\text{th}}$ datapoint and $y^{(n)}$ is the corresponding class.

Study the effect of the interactive plot parameter values to the visualization. Once you are happy with their values, execute also the cell below it to extract the data from the interactive cell. In the 'Answer'-section, briefly describe in the empty cell i) how one could expect to see differences, ii) differences observed if any, and iii) the parameter values that you chose.

In [ ]:
interactive_plot2 = split_dataset_and_plot_objective_interactively(positives, num_positives, negatives, num_negatives)
interactive_plot2

In [ ]:
training_inputs, training_targets, num_training_examples, num_training_positives, num_training_negatives, \
           validation_inputs, validation_targets, num_validation_examples, num_validation_positives, num_validation_negatives, \
           test_inputs, test_targets, num_test_examples, num_test_positives, num_test_negatives, param1_meshgrid_values, \
           param2_meshgrid_values, training_costs_on_meshgrid, validation_costs_on_meshgrid = interactive_plot2.result

### Answer
Input your answers to the cell below.

## Task 3: Optimization of model parameter values
Executing the cell below creates an interactive plot to optimize model parameters.

The model parameters $\theta_1$ and $\theta_2$ are updated using *stochastic* gradient descent (with momentum) on the negative log-likelihood of the training data. At each parameter update, the true gradient is estimated using a mini-batch of the (training) data instead of calculated with the full (training) dataset; hence *stochastic*. Best model parameters in the optimization here are in fact not decided based on the negative log-likelihood, but based on a different metric (which is typical), classification accuracy for simplicity; the metric is non-differentiable and we cannot use gradient-based optimization for directly optimizing it as is the case with the negative log-likelihood objective here. The threshold $T$ for the classification assuming specific model parameter values, is obtained by considering various threshold values in classification and selecting the one with which maximizes the classification accuracy, on the validation set; the calculation is done at every iteration, for the updated parameter values. 

The configuration of model parameter values $(\theta_1, \theta_2)$ that one obtains using the optimization scheme are 
the ones in the SGD (+momentum) iterative update process that resulted in the highest value of the classification accuracy.

Properties that can be changed in the optimization:
* initial_parameter_values: the initial parameter values; keep the initial ones for your final results 
* num_iterations: amount of SGD (with momentum) iterations 
* minibatch_size: amount of examples in a minibatch of data; this is used to calculate the gradient estimate
* stratified_minibatch: if checked, then each minibatch of data assumes (roughly) equal amount of positives and negatives; otherwise sampling of the examples from the training data to the minibatch ignores the class and the class rates vary across different minibatches of data.
* learning_rate: the learning rate
* momentum_rate: the momentum rate

Press the 'Run Interact'-button to run the optimization; different runs with same interaction parameter values (are expected to) result in different outcomes due to the stochasticity in the minibatch sampling. Study the effect of the interaction parameter values. Try then to the find such values for which you obtain good training and validation data classification (assuming your the data), and ideally also in terms of the negative log-likelihood on the training and validation data. The current best parameter values (assuming our validation data classification accuracy criterion) are highlighted with a green cross in the contour-plots on the right. Aim to land the final green crosses near the inner-most contours, like inside. Also, try to get a stable update process.
Once done, execute the cell below the plots to get the obtained model parameter values from the interactive plot instance.

In the 'Answer'-section, briefly describe in the empty cell i) how does the effect of changing the minibatch size can be seen from the plots, ii) observations on the process of adjusting the interaction parameter values to get to the target, and iii)
the parameter values that you chose.

In [ ]:
interactive_plot3 = create_interactive_training_plot(param1_meshgrid_values, param2_meshgrid_values, training_costs_on_meshgrid,
                   validation_costs_on_meshgrid, training_inputs, training_targets, num_training_examples,
                   num_training_positives, num_training_negatives, validation_inputs, validation_targets,
                   num_validation_examples)
interactive_plot3

In [ ]:
parameter_values, probability_threshold = interactive_plot3.result
print(parameter_values)
print(probability_threshold)

### Answer
Input your answers to the cell below.

## Task 4: Model testing (and tweaking)
Executing the cell below calculates classification accuracies with the obtained model parameters and the associated probability 
threshold for prediction.

Executing the cell below it, creates an interactive plot to study the effect of changing the probability threshold ($T$) in classifying validation data.

Executing the cell further below recalculates the classification accuracies with the probability threshold chosen in the interactive plot.

Vary the probability threshold (utilizing the interactive plot) and observe the effects; the model should be run on the test set in practice only once, but here we can do a bit of studying for understanding the behaviour on the test data as well.

In the 'Answer'-section, briefly describe in the empty cell i) the effect of changing the probability threshold (to the model and to the predictions), ii) does the threshold need to be adjusted to get no false negatives (so no positive example is classified as negative, and so no triangles with blue fill and black border) under the validation data and if so, what is the effect of the adjustment to the classification accuracies, iii) original and chosen probability threshold.

In [ ]:
calculate_classification_accuracies(parameter_values, probability_threshold, training_inputs, training_targets,
                                        validation_inputs, validation_targets, test_inputs, test_targets)

In [ ]:
interactive_plot4 = create_interactive_inference_plot(validation_inputs, parameter_values, probability_threshold, validation_targets)
interactive_plot4

In [ ]:
updated_probability_threshold = interactive_plot4.result
calculate_classification_accuracies(parameter_values, updated_probability_threshold, training_inputs, training_targets,
                                        validation_inputs, validation_targets, test_inputs, test_targets)

### Answer
Input your answers to the cell below.